In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import Tensor
import numpy as np

In [3]:
from torchvision.datasets import MNIST
from torchvision import transforms

composed_transforms = transforms.Compose([
    transforms.ToTensor(), # performs scaling by default for image datasets between range(0-1)
    transforms.Normalize((0.5,), (0.5,))
])

train_val_set = MNIST("./data/train", train=True, transform=composed_transforms, download=True)
test_set = MNIST("./data/test", train=False, transform=composed_transforms, download=True)

train_set, val_set = torch.utils.data.random_split(dataset=train_val_set, lengths=[.9, .1])


### Note 
```
If iteration performed over a dataset with transforms, transform operations cen be triggered, if not, will be triggered on train loop and may slow down the process
```

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [17]:
X, y = iter(train_set).__next__()
print(X.shape, y)
print(X[X != 0]) # already scaled between 0-1
transforms.ToPILImage()(X)

torch.Size([1, 28, 28]) 8
tensor([-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.000

In [18]:
from torch import nn

class LeNet5(nn.Module):
    
    def __init__(self) -> None:
        super().__init__()
        self.feature_extractor = nn.Sequential(
            # Input -> (batch_size,1,32,32)
            # Layer 1
            nn.Conv2d(
                in_channels=1, 
                out_channels=6,
                kernel_size=5,
                padding=2, # padding as 2 for achieving expected size
                stride=1), # -> (batch_size,6,28,28)
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2), # -> (batch_size,6,14,14)
            
            # Layer 2
            nn.Conv2d(
                in_channels=6,
                out_channels=16,
                kernel_size=5,
                stride=1,
                padding=0), # -> (batch_size,16,10,10)
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2), # (batch_size,16,5,5)
        )
        self.fully_connected = nn.Sequential(
            nn.Flatten(), # (batch_size,400)
            nn.Linear(in_features=400, out_features=120), # (batch_size,120)
            nn.Tanh(),
            nn.Linear(in_features=120, out_features=84), # (batch_size,84)
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=10), # (batch_size,10)
        )
    
    def forward(self, x):
        return self.fully_connected(self.feature_extractor(x))
    
    
        

In [19]:
model = LeNet5().to(device)
model

LeNet5(
  (feature_extractor): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (fully_connected): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=400, out_features=120, bias=True)
    (2): Tanh()
    (3): Linear(in_features=120, out_features=84, bias=True)
    (4): Tanh()
    (5): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [20]:
from torchinfo import summary



summary(model, input_size=(1,1,28,28))

Layer (type:depth-idx)                   Output Shape              Param #
LeNet5                                   [1, 10]                   --
├─Sequential: 1-1                        [1, 16, 5, 5]             --
│    └─Conv2d: 2-1                       [1, 6, 28, 28]            156
│    └─Tanh: 2-2                         [1, 6, 28, 28]            --
│    └─AvgPool2d: 2-3                    [1, 6, 14, 14]            --
│    └─Conv2d: 2-4                       [1, 16, 10, 10]           2,416
│    └─Tanh: 2-5                         [1, 16, 10, 10]           --
│    └─AvgPool2d: 2-6                    [1, 16, 5, 5]             --
├─Sequential: 1-2                        [1, 10]                   --
│    └─Flatten: 2-7                      [1, 400]                  --
│    └─Linear: 2-8                       [1, 120]                  48,120
│    └─Tanh: 2-9                         [1, 120]                  --
│    └─Linear: 2-10                      [1, 84]                   10,164
│  

In [ ]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.utils.tensorboard import SummaryWriter
import os
from datetime import datetime

learning_rate = 0.001
epochs = 10
batch_size=16

# Should be SGD according to paper
optimizer = Adam(params=model.parameters(), lr=learning_rate)
criterion =  CrossEntropyLoss()

summary_logger = SummaryWriter(
    os.path.join(
        "experiments",
        datetime.now().strftime("%d.%m.%Y"),
        "Experiment 1",
        "LeNet5_01"
    )
)


In [22]:
train_dataloader = DataLoader(
    train_set, 
    batch_size=batch_size,
    shuffle=True,
    num_workers=8)

val_dataloader = DataLoader(
    val_set, 
    batch_size=batch_size,
    shuffle=True,
    num_workers=8)

test_dataloader = DataLoader(
    test_set, 
    batch_size=batch_size,
    shuffle=True,
    num_workers=8)

train_dataloader, val_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x1c55391f750>,
 <torch.utils.data.dataloader.DataLoader at 0x1c512ef49d0>)

In [23]:
from torchmetrics import Accuracy


accuracy = Accuracy(task='multiclass', num_classes=10).to(device)


def calculate_accuracy(pred: Tensor, y: Tensor) -> Tensor:
    y_pred = nn.functional.softmax(pred, dim=1)
    argmax_ = torch.argmax(y_pred, axis=1)
    trues = (argmax_ == y).float()
    return torch.mean(trues)



In [ ]:
from tqdm import tqdm 

# MNIST well balanced dataset, no need for precision or recall


# rain_losses = list()
# alidation_losses = list()

# rain_accuracies = list()
# alidation_accuracies = list()

# pre calculate length of dataloaders
train_dataloader_length = len(train_dataloader)
val_dataloader_length = len(val_dataloader)
print("Training...")
for epoch in tqdm(range(epochs)):
    
    # init losses for this epoch
    train_loss, train_accuracy = 0.0,0.0
    # train loop
    for X_train, y_train in tqdm(train_dataloader):
        # activate train mode, not batchNorm or Dropout for this model but convention
        model.train()
        
        # to gpu if available
        X_train = X_train.to(device)
        y_train = y_train.to(device)
        
        # zeroing gradients
        optimizer.zero_grad()
        
        # prediction
        pred = model(X_train)
        
        
        # calculation of loss
        loss = criterion(pred, y_train)
        train_loss += loss
        
        acc = accuracy(pred, y_train)
        train_accuracy+=acc
        
        # print("*******")        
        # backpropagation operation over loss
        loss.backward()
        
        # update params according to optimizer algorithm and model parameters
        optimizer.step()        
    
    # average loss and accuracy on batch
    train_loss /= train_dataloader_length
    train_accuracy /= train_dataloader_length
        
        
    
    # validation loop
    validation_loss, validation_accuracy = 0.0,0.0
    model.eval()
    print("Validation...")
    with torch.inference_mode():
        for X_validation, y_validation in tqdm(val_dataloader):
        
            # to gpu if available
            X_validation = X_validation.to(device)
            y_validation = y_validation.to(device)
            
            # prediction
            pred = model(X_validation)
            
            # calculating loss for val
            loss = criterion(pred, y_validation)
            validation_loss += loss
            
            # calculate accuracy using lightning metrics and add for current epoch
            acc = accuracy(pred, y_validation)
            validation_accuracy+=acc
            
        # average loss and accuracy on batch
        validation_loss/=val_dataloader_length
        validation_accuracy/=val_dataloader_length
        
    # SummaryWriter for losses
    summary_logger.add_scalars(
        main_tag="Losses",
        tag_scalar_dict={
            "train/loss": train_loss,
            "validation/loss": validation_loss
        },
        global_step=epoch
    )
    # SummaryWriter for accuracies
    summary_logger.add_scalars(
        main_tag="Accuracies",
        tag_scalar_dict={
            "train/accuracy": train_accuracy,
            "validation/accuracy": validation_accuracy
        },
        global_step=epoch
    )
    
    print(f" Epoch: {epoch} -- Train Loss: {train_loss: .3f} -- Train Acc : {train_accuracy: .3f} -- Val Loss: {validation_loss: .3f} -- Val Acc: {validation_accuracy: .3f} ")


Train...


100%|██████████| 3375/3375 [00:41<00:00, 80.46it/s] 


Validation...


 10%|█         | 1/10 [00:48<07:15, 48.37s/it]

 Epoch: 0 -- Train Loss:  0.201 -- Train Acc :  0.940 -- Val Loss:  0.110 -- Val Acc:  0.967 


...

 Epoch: 8 -- Train Loss:  0.025 -- Train Acc :  0.992 -- Val Loss:  0.061 -- Val Acc:  0.982 


100%|██████████| 3375/3375 [00:40<00:00, 83.06it/s] 


Validation...


100%|██████████| 10/10 [07:51<00:00, 47.14s/it]

 Epoch: 9 -- Train Loss:  0.023 -- Train Acc :  0.993 -- Val Loss:  0.055 -- Val Acc:  0.984 


In [56]:
# test 



test_dataloader_length = len(test_dataloader)

model.eval()
test_loss, test_accuracy = 0.0,0.0
with torch.inference_mode():
    for X_test, y_test in tqdm(test_dataloader):
        
        X_test = X_test.to(device) 
        y_test = y_test.to(device) 
        
        # test prediction
        pred = model(X_test)
        
        # calculating loss for test set
        loss = criterion(pred, y_test)
        test_loss += loss
        
        # calculate accuracy using lightning metrics and add for current epoch
        acc = accuracy(pred, y_test)
        test_accuracy+=acc

        
    # average loss and accuracy on batch
    test_loss/=test_dataloader_length
    test_accuracy/=test_dataloader_length
    
print(f" Test Loss: {test_loss: .3f} -- Test Acc : {test_accuracy: .3f}")

# SummaryWriter for losses
summary_logger.add_scalars(
    main_tag="Losses",
    tag_scalar_dict={
        "test/loss": test_loss,
    },
    global_step=epoch
)

# SummaryWriter for accuracies
summary_logger.add_scalars(
    main_tag="Accuracies",
    tag_scalar_dict={
        "test/accuracy": test_accuracy
    },
    global_step=epoch
)

        

100%|██████████| 625/625 [00:07<00:00, 87.37it/s] 

 Test Loss:  0.060 -- Test Acc :  0.985


In [57]:
def save_model(model:nn.Module, epochs: int, base_path: str, name: str):
    torch.save(model, os.path.join(base_path, f"{name}_{epochs}.pth"))

In [60]:
save_model(model, epochs, "models", "LeNet5_model_01")

In [86]:
int(torch.randint(0, 100, (1,1))[0][0])

18

In [82]:
test_X, test_y = iter(test_dataloader).__next__()

In [84]:
test_X.shape

torch.Size([16, 1, 28, 28])

In [90]:
out = model(test_X[:2].to(device))

print(out)
print(test_y[:2])


tensor([[-10.3349,   1.7753,  -6.6977,  -2.3191,  15.3755,  -2.7241,  -5.7152,
           1.6385,  -1.7889,   4.9020],
        [  2.6413,  -1.0144,   1.3535, -11.1045,   3.7988,   0.8837,  18.9129,
          -4.9064,   2.4326,  -3.1384]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor([4, 6])


In [3]:
%reload_ext tensorboard
%tensorboard --logdir={"./experiments"}

Reusing TensorBoard on port 6007 (pid 14136), started 0:00:07 ago. (Use '!kill 14136' to kill it.)

In [8]:
!taskkill /F /PID  14136

SUCCESS: The process with PID 14136 has been terminated.


In [ ]:
# TODO: Add Normalization